In [10]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from torch.nn import BCELoss, MSELoss, Sigmoid
from torch import optim
from torch.nn import Conv2d

from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy


import warnings

warnings.filterwarnings("ignore")

sys.path.insert(0, "/appli")


plt.rcParams["figure.figsize"] = (15, 15)
from utile.pre_processing import PreProcessingPipeline

from utile.loaders import load_file, load_image, create_batch, RNSADataset
import cv2
from utile.plot import plot_sample

#%matplotlib widget
from opencv_transforms import transforms

seed = 53

device = "cuda" if torch.cuda.is_available() else "cpu"

print("PyTorch Version: ", torch.__version__)
print("Torchvision Version: ", torchvision.__version__)

PyTorch Version:  1.14.0a0+410ce96
Torchvision Version:  0.15.0a0


In [11]:
# Import train data
img_path = "../kaggle_dataset"
train_df = pd.read_csv("data/train.csv")

In [12]:
efficientnet = torch.hub.load(
    "NVIDIA/DeepLearningExamples:torchhub", "nvidia_efficientnet_b0", pretrained=True
)

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [13]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "../kaggle_dataset"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 1

loss = BCELoss()
# loss = MSELoss()

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

optimizer = optim.Adam(efficientnet.parameters(), lr=0.0001)

In [14]:
transformed_dataset = RNSADataset(
    root_dir="../kaggle_dataset",
    csv_file="data/train.csv",
    transform=transforms.Compose([PreProcessingPipeline()]),
)

In [15]:
dataloader = DataLoader(
    transformed_dataset, batch_size=batch_size, shuffle=True, num_workers=4
)

In [16]:
# efficientnet.fc = nn.Linear(512, num_classes)
efficientnet.classifier.fc = nn.Sequential(
    nn.Linear(in_features=1280, out_features=512, bias=True),
    nn.LeakyReLU(),
    nn.Linear(512, 128),
    nn.LeakyReLU(),
    nn.Linear(128, 64),
    nn.LeakyReLU(),
    nn.Linear(64, 8),
    nn.LeakyReLU(),
    nn.Linear(8, 1),
    nn.Sigmoid(),
)


efficientnet.stem.conv = Conv2d(
    1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False
)

In [17]:
def train_model(
    model, dataloaders, criterion, optimizer, num_epochs=15, is_inception=False
):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs - 1))
        print("-" * 10)

        # Each epoch has a training and validation phase
        """
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
        """
        running_loss = 0.0
        running_corrects = 0
        phase = "train"
        # Iterate over data.
        for elem in dataloaders:
            inputs = elem["image"]
            labels = elem["labels"]

            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == "train"):
                # Get model outputs and calculate loss
                # Special case for inception because in training it has an auxiliary output. In train
                #   mode we calculate the loss by summing the final output and the auxiliary output
                #   but in testing we only consider the final output.
                # target = labels.unsqueeze(1)
                if is_inception and phase == "train":
                    # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958

                    outputs, aux_outputs = model(inputs)
                    loss1 = criterion(outputs, labels)
                    loss2 = criterion(aux_outputs, labels)
                    loss = loss1 + 0.4 * loss2
                else:
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)

                # backward + optimize only if in training phase
                if phase == "train":
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            # epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_loss = running_loss / len(dataloaders.dataset)

            # epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders.dataset)

            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == "val":
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print(
        "Training complete in {:.0f}m {:.0f}s".format(
            time_elapsed // 60, time_elapsed % 60
        )
    )
    print("Best val Acc: {:4f}".format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [18]:
train_model(
    model=efficientnet,
    dataloaders=dataloader,
    criterion=loss,
    optimizer=optimizer,
    num_epochs=num_epochs,
)

Epoch 0/0
----------
train Loss: 0.0001 Acc: 0.0010
train Loss: 0.0002 Acc: 0.0022
train Loss: 0.0004 Acc: 0.0034
train Loss: 0.0005 Acc: 0.0045
train Loss: 0.0006 Acc: 0.0057
train Loss: 0.0007 Acc: 0.0069
train Loss: 0.0008 Acc: 0.0080


KeyboardInterrupt: 

In [ ]:
utils = torch.hub.load(
    "NVIDIA/DeepLearningExamples:torchhub", "nvidia_convnets_processing_utils"
)

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
